In [9]:
import pandas as pd
import yaml
import utils
import seaborn as sns
import matplotlib.pyplot as plt

### Load data

In [10]:
# Read IDs to access the correct doc/sheet:
with open("config.yml") as file:
    cfg_data = yaml.safe_load(file)
cfg_data

{'DOCID': '1G0FmJhkOME_sv66hWmhnZS5qR2KMTY7nzkxksv46bfk',
 'GID': {'2022': '203339396', '2023': '453658888', '2024': '491268892'}}

In [11]:
# Load data:
YEARS_TO_LOAD = [2023, 2024]  

df_raw = pd.DataFrame({})

for year in YEARS_TO_LOAD:  
    
    # Load data for given year:
    df_i = utils.load_data(year=year, cfg=cfg_data)
    df_i["year"] = year

    # Store in dataframe:
    df_raw = pd.concat([df_raw, df_i], ignore_index=True)

print(df_raw.shape)
df_raw.head()

(2028, 31)


,Timestamp,What country do you work in?,Where is the closest major city or hub?,Biotech sub industry?,Company or Institution Name,Company Details - public/private/start-up/ subsidiary of,Company Detail - Approximate Company Size,Role / Title of current position,Highest achieved Formal Education,List other relevant and recognized certifications,...,[OPTIONAL] Sign on - Relocation Assistance Total Value,[Optional] Company Review,year,Where are you located?,What degrees do you have?,Compensation - Annual Equity/Stock Option,Compensation - Sign on Stock/Equity Options,Compensation - Retirement Percent Match (free text),"[Optional] Work Life Balance - On average, how many hours do you work per week",Survey Feedback
0,8/6/2023 13:35:24,Canada,Toronto,Academia/Research/Institution,Prefer not to say,Private,200-1000,Medical Writer,Bachelors or Equivalent,NaN,...,NaN,NaN,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4/19/2023 19:24:24,Germany,Munich,Academia/Research/Institution,Max Planck Institute,Academia/College/Universities,5000+,Post-doc,PhD or Equivalent,NaN,...,NaN,"Very good place, very fair treatment",2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6/22/2023 16:28:50,Slovenia,Ljubljana,Academia/Research/Institution,Prefer not to say,Public,50-200,Student assistant researcher,Bachelors or Equivalent,NaN,...,NaN,NaN,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6/14/2023 15:30:45,United Kingdom,Edinburgh,Academia/Research/Institution,University of Edinburgh,Academia/College/Universities,5000+,Postdoc,PhD or Equivalent,NaN,...,NaN,NaN,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3/29/2023 2:06:32,United States,Bay Area,Academia/Research/Institution,Prefer not to say,Start-up,1-50,Research Scientist,Masters or Equivalent,NaN,...,NaN,NaN,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
print(df_raw.columns)

Index(['Timestamp', 'What country do you work in?',
       'Where is the closest major city or hub?', 'Biotech sub industry?',
       'Company or Institution Name',
       'Company Details - public/private/start-up/ subsidiary of ',
       'Company Detail - Approximate Company Size',
       'Role / Title of current position', 'Highest achieved Formal Education',
       'List other relevant and recognized certifications',
       '[Optional] Briefly describe your position', 'Years of Experience',
       'Compensation - Annual Base Salary/Pay', 'Compensation - Overtime Pay',
       'Compensation - Annual Target Bonus ($)',
       'Compensation - Annual Equity', 'Compensation - Sign on Bonus Value',
       'Compensation - Stock Options',
       'Compensation - Retirement Benefits (free text)',
       'Compensation - Healthcare Benefits (free text)',
       'Compensation - Most recent annual yearly raise (%)',
       '[OPTIONAL] Sign on - Relocation Assistance Total Value',
       '[Optiona

### Data cleanup

In [13]:
df = df_raw.copy()

In [14]:
df.columns

Index(['Timestamp', 'What country do you work in?',
       'Where is the closest major city or hub?', 'Biotech sub industry?',
       'Company or Institution Name',
       'Company Details - public/private/start-up/ subsidiary of ',
       'Company Detail - Approximate Company Size',
       'Role / Title of current position', 'Highest achieved Formal Education',
       'List other relevant and recognized certifications',
       '[Optional] Briefly describe your position', 'Years of Experience',
       'Compensation - Annual Base Salary/Pay', 'Compensation - Overtime Pay',
       'Compensation - Annual Target Bonus ($)',
       'Compensation - Annual Equity', 'Compensation - Sign on Bonus Value',
       'Compensation - Stock Options',
       'Compensation - Retirement Benefits (free text)',
       'Compensation - Healthcare Benefits (free text)',
       'Compensation - Most recent annual yearly raise (%)',
       '[OPTIONAL] Sign on - Relocation Assistance Total Value',
       '[Optiona

##### Column names

In [15]:
repl_cols = {#'Timestamp',
             'What country do you work in?': "country_1",
             'Where are you located?': "country_2",
             'Where is the closest major city or hub?': "city",
             
             'Biotech sub industry?': "company_field",
             'Company or Institution Name': "company_name",
             'Company Details - public/private/start-up/ subsidiary of ': "company_type",
             'Company Detail - Approximate Company Size': "company_size",
             
             'Role / Title of current position': "role",
             
             'What degrees do you have? ': "degrees_1",
             "Highest achieved Formal Education": "degrees_2",
             
            #'List other relevant and recognized certifications',
            #'[Optional] Briefly describe your position',
            
            'Years of Experience': "experience_yrs",
            'Compensation - Annual Base Salary/Pay': "pay_annual_base",
            'Compensation - Overtime Pay': "pay_overtime",
            'Compensation - Annual Target Bonus ($)': "pay_annual_bonus",
            'Compensation - Annual Equity/Stock Option': "pay_annual_equity",
            'Compensation - Most recent annual yearly raise (%)': "pay_raise",
            
            # 'Compensation - Sign on Bonus Value',
            #'Compensation - Sign on Stock/Equity Options',
            #'Compensation - Retirement Benefits (free text)',
            #'Compensation - Retirement Percent Match (free text)',
            #'Compensation - Healthcare Benefits (free text)',
            #'[OPTIONAL] Sign on - Relocation Assistance Total Value',
            #'[Optional] Company Review',
            #'[Optional] Work Life Balance - On average, how many hours do you work per week',
            #'Survey Feedback'
            }

df.rename(columns=repl_cols, inplace=True)
df.head()

,Timestamp,country_1,city,company_field,company_name,company_type,company_size,role,degrees_2,List other relevant and recognized certifications,...,[OPTIONAL] Sign on - Relocation Assistance Total Value,[Optional] Company Review,year,country_2,degrees_1,pay_annual_equity,Compensation - Sign on Stock/Equity Options,Compensation - Retirement Percent Match (free text),"[Optional] Work Life Balance - On average, how many hours do you work per week",Survey Feedback
0,8/6/2023 13:35:24,Canada,Toronto,Academia/Research/Institution,Prefer not to say,Private,200-1000,Medical Writer,Bachelors or Equivalent,NaN,...,NaN,NaN,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4/19/2023 19:24:24,Germany,Munich,Academia/Research/Institution,Max Planck Institute,Academia/College/Universities,5000+,Post-doc,PhD or Equivalent,NaN,...,NaN,"Very good place, very fair treatment",2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6/22/2023 16:28:50,Slovenia,Ljubljana,Academia/Research/Institution,Prefer not to say,Public,50-200,Student assistant researcher,Bachelors or Equivalent,NaN,...,NaN,NaN,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6/14/2023 15:30:45,United Kingdom,Edinburgh,Academia/Research/Institution,University of Edinburgh,Academia/College/Universities,5000+,Postdoc,PhD or Equivalent,NaN,...,NaN,NaN,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3/29/2023 2:06:32,United States,Bay Area,Academia/Research/Institution,Prefer not to say,Start-up,1-50,Research Scientist,Masters or Equivalent,NaN,...,NaN,NaN,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Years of experience

In [16]:
df["experience_yrs"].unique()

array(['5', '4', '0', '2', '7', '1', '13', '3', '20', '6', '9', '12', '8',
       '17', '11', '10', '14', '15', '16', '24', '22', '18', '21', '25',
       '30 or more', '23', '19', 10, 0, 7, 27, 2, 4, 17, 15, 8, 6, 3, 1,
       14, 5, 9, 12, 18, 11, 13, 23, 20, 21, 19, 16, 32, 25, 24, 22, 30],
      dtype=object)

In [17]:
repl_exp = {"30 or more": "30"}

df = utils.value_replacement(df, repl_exp, column="experience_yrs")
df["experience_yrs"].unique()

array(['5', '4', '0', '2', '7', '1', '13', '3', '20', '6', '9', '12', '8',
       '17', '11', '10', '14', '15', '16', '24', '22', '18', '21', '25',
       '30', '23', '19', nan], dtype=object)

In [18]:
# Convert to numerical:
df["experience_yrs"] = df["experience_yrs"].astype(float)

##### Educational background

In [19]:
df["degrees"] = df["degrees_1"].fillna(df["degrees_2"])
df.head()

,Timestamp,country_1,city,company_field,company_name,company_type,company_size,role,degrees_2,List other relevant and recognized certifications,...,[Optional] Company Review,year,country_2,degrees_1,pay_annual_equity,Compensation - Sign on Stock/Equity Options,Compensation - Retirement Percent Match (free text),"[Optional] Work Life Balance - On average, how many hours do you work per week",Survey Feedback,degrees
0,8/6/2023 13:35:24,Canada,Toronto,Academia/Research/Institution,Prefer not to say,Private,200-1000,Medical Writer,Bachelors or Equivalent,NaN,...,NaN,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bachelors or Equivalent
1,4/19/2023 19:24:24,Germany,Munich,Academia/Research/Institution,Max Planck Institute,Academia/College/Universities,5000+,Post-doc,PhD or Equivalent,NaN,...,"Very good place, very fair treatment",2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PhD or Equivalent
2,6/22/2023 16:28:50,Slovenia,Ljubljana,Academia/Research/Institution,Prefer not to say,Public,50-200,Student assistant researcher,Bachelors or Equivalent,NaN,...,NaN,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bachelors or Equivalent
3,6/14/2023 15:30:45,United Kingdom,Edinburgh,Academia/Research/Institution,University of Edinburgh,Academia/College/Universities,5000+,Postdoc,PhD or Equivalent,NaN,...,NaN,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PhD or Equivalent
4,3/29/2023 2:06:32,United States,Bay Area,Academia/Research/Institution,Prefer not to say,Start-up,1-50,Research Scientist,Masters or Equivalent,NaN,...,NaN,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Masters or Equivalent


In [20]:
# Educational background:
repl_education = {" or Equivalent": "",
                  "Bachelors": "BSc",
                  "Masters": "MSc",
                  "High School": "HS",
                  "Associate’s Degree": "AD"
                  }

df = utils.value_replacement(df, repl_education, column="degrees")
df["degrees"].unique()

array(['BSc', 'PhD', 'MSc', 'AD', 'M.D./PharmD/D.D.S.', 'HS', 'BSc, MSc',
       'HS, BSc, MSc, PhD', 'HS, AD, BSc', 'BSc, PhD', 'BSc, MSc, PhD',
       'HS, BSc, MSc', 'HS, BSc', 'BSc, MSc, MD', 'HS, AD, BSc, MSc',
       'MD', 'HS, BSc, PhD', 'MSc, PhD', 'PharmD', 'HS, BSc, PhD, MD',
       'HS, BSc, PhD, JD', 'HS, PharmD', 'MSc, PharmD', 'JD', 'HS, AD',
       'HS, AD, BSc, PhD', 'AD, BSc', 'PhD, MD', 'HS, AD, BSc, MSc, PhD',
       'BSc, PharmD', 'PhD, PharmD', 'HS, MSc, PhD', 'HS, BSc, PharmD'],
      dtype=object)

In [21]:
# Test one-hot encoding:
df["degrees"].str.get_dummies(sep=", ").add_prefix("edu_")

,edu_AD,edu_BSc,edu_HS,edu_JD,edu_M.D./PharmD/D.D.S.,edu_MD,edu_MSc,edu_PhD,edu_PharmD
0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
2023,0,0,0,0,0,1,0,0,0
2024,0,1,0,0,0,0,0,0,0
2025,0,0,0,0,0,0,0,1,0
2026,0,1,0,0,0,0,0,0,0


In [22]:
# Replace column with one-hot encodings:
df = pd.concat([df, df["degrees"].str.get_dummies(sep=", ").add_prefix("edu_")], axis = 1).drop("degrees", axis=1)
df.head()

,Timestamp,country_1,city,company_field,company_name,company_type,company_size,role,degrees_2,List other relevant and recognized certifications,...,Survey Feedback,edu_AD,edu_BSc,edu_HS,edu_JD,edu_M.D./PharmD/D.D.S.,edu_MD,edu_MSc,edu_PhD,edu_PharmD
0,8/6/2023 13:35:24,Canada,Toronto,Academia/Research/Institution,Prefer not to say,Private,200-1000,Medical Writer,Bachelors or Equivalent,NaN,...,NaN,0,1,0,0,0,0,0,0,0
1,4/19/2023 19:24:24,Germany,Munich,Academia/Research/Institution,Max Planck Institute,Academia/College/Universities,5000+,Post-doc,PhD or Equivalent,NaN,...,NaN,0,0,0,0,0,0,0,1,0
2,6/22/2023 16:28:50,Slovenia,Ljubljana,Academia/Research/Institution,Prefer not to say,Public,50-200,Student assistant researcher,Bachelors or Equivalent,NaN,...,NaN,0,1,0,0,0,0,0,0,0
3,6/14/2023 15:30:45,United Kingdom,Edinburgh,Academia/Research/Institution,University of Edinburgh,Academia/College/Universities,5000+,Postdoc,PhD or Equivalent,NaN,...,NaN,0,0,0,0,0,0,0,1,0
4,3/29/2023 2:06:32,United States,Bay Area,Academia/Research/Institution,Prefer not to say,Start-up,1-50,Research Scientist,Masters or Equivalent,NaN,...,NaN,0,0,0,0,0,0,1,0,0


In [23]:
# Assign the highest level of education
# Note: MD/JD/etc are classified as "Others", as they can coexist with other degrees

order_education = ["HS", "BSc", "MSc", "PhD"]   # order matters here!


df["edu_highest"] = "Other"

for deg in order_education:
    df.loc[df[f"edu_{deg}"] == 1, "edu_highest"] = deg
df

,Timestamp,country_1,city,company_field,company_name,company_type,company_size,role,degrees_2,List other relevant and recognized certifications,...,edu_AD,edu_BSc,edu_HS,edu_JD,edu_M.D./PharmD/D.D.S.,edu_MD,edu_MSc,edu_PhD,edu_PharmD,edu_highest
0,8/6/2023 13:35:24,Canada,Toronto,Academia/Research/Institution,Prefer not to say,Private,200-1000,Medical Writer,Bachelors or Equivalent,NaN,...,0,1,0,0,0,0,0,0,0,BSc
1,4/19/2023 19:24:24,Germany,Munich,Academia/Research/Institution,Max Planck Institute,Academia/College/Universities,5000+,Post-doc,PhD or Equivalent,NaN,...,0,0,0,0,0,0,0,1,0,PhD
2,6/22/2023 16:28:50,Slovenia,Ljubljana,Academia/Research/Institution,Prefer not to say,Public,50-200,Student assistant researcher,Bachelors or Equivalent,NaN,...,0,1,0,0,0,0,0,0,0,BSc
3,6/14/2023 15:30:45,United Kingdom,Edinburgh,Academia/Research/Institution,University of Edinburgh,Academia/College/Universities,5000+,Postdoc,PhD or Equivalent,NaN,...,0,0,0,0,0,0,0,1,0,PhD
4,3/29/2023 2:06:32,United States,Bay Area,Academia/Research/Institution,Prefer not to say,Start-up,1-50,Research Scientist,Masters or Equivalent,NaN,...,0,0,0,0,0,0,1,0,0,MSc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023,9/29/2024 21:11:18,NaN,NaN,Big Pharmaceuticals,Prefer not to say,Public,5000+,Asset Exec,NaN,NaN,...,0,0,0,0,0,1,0,0,0,Other
2024,9/29/2024 22:12:46,NaN,NaN,Contraceptives,Mucommune,Start-up,1-50,Scientist,NaN,NaN,...,0,1,0,0,0,0,0,0,0,BSc
2025,9/30/2024 5:33:54,NaN,NaN,Agriculture,Syngenta,Private,5000+,Principal Scientist I,NaN,NaN,...,0,0,0,0,0,0,0,1,0,PhD
2026,9/30/2024 8:01:19,NaN,NaN,Big Pharmaceuticals,Regeneron,Public,5000+,Associate Scientist,NaN,NaN,...,0,1,0,0,0,0,0,0,0,BSc


##### Country

In [24]:
# Country:
repl_country = {"united states of america": "us",
                "united stated of americ": "us",             
                "united states": "us",
                "united stated": "us",
                "usa": "us",
                "us - florida": "us",
                "us (remote)": "us",
                "u.s.a.": "us",
                "united kingdom": "uk",
                "england": "uk",
                "netherlands": "benelux",
                
               }

# Clean names:
df["country_1"] = df["country_1"].str.lower().str.rstrip(" ")
print(df["country_1"].unique())

df = utils.value_replacement(df, replacement=repl_country, column="country_1")
print(df["country_1"].unique())

['canada' 'germany' 'slovenia' 'united kingdom' 'united states' 'usa'
 'united states of america' 'austria' 'united stated of americ' 'us'
 'india' 'uk' 'england' 'ireland' 'netherlands' 'united states - florida'
 'america' 'australia' 'portugal' 'sweden' 'china' 'greece' 'hungary'
 'u.s.a.' 'united states (remote)' 'united stated' 'switzerland' nan]
['canada' 'germany' 'slovenia' 'uk' 'us' 'austria' 'india' 'ireland'
 'benelux' 'america' 'australia' 'portugal' 'sweden' 'china' 'greece'
 'hungary' 'switzerland' nan]


In [25]:
repl_country2 = {"west coast (california & pacific northwest)": "us",   
                 "dc metro area (dc, va, md, de)": "us",             
                 "pharma central (ny, nj, pa)": "us",
                 "carolinas & southeast (from nc to ar, south fl and la)": "us",
                 "new england (ma, ct, ri, nh, vt, me)": "us",
                 "midwest (from oh to ks, north to nd)": "us",
                 "south & mountain west (tx to az, north to mt)": "us",
                 "other us location (hi, ak, pr, etc.)": "us",
                 "co": "us",
                 "san diego, ca": "us",
                 "remote - us": "us",
                 "research triangle park, nc": "us",
                 "boston": "us",
                 "tennessee": "us",
                 "nl": "benelux",
                 "belgium": "benelux",
                 "netherlands": "benelux",
                 "united kingdom and ireland": "uk"
                }

# Clean names:
df["country_2"] = df["country_2"].str.lower().str.rstrip(" ")
print(df["country_2"].unique())

df = utils.value_replacement(df, replacement=repl_country2, column="country_2")
print(df["country_2"].unique())

[nan 'canada' 'west coast (california & pacific northwest)' 'france'
 'pharma central (ny, nj, pa)' 'united kingdom and ireland' 'denmark'
 'germany' 'carolinas & southeast (from nc to ar, south fl and la)'
 'new england (ma, ct, ri, nh, vt, me)' 'dc metro area (dc, va, md, de)'
 'spain' 'midwest (from oh to ks, north to nd)' 'belgium' 'eu'
 'south & mountain west (tx to az, north to mt)' 'india'
 'other us location (hi, ak, pr, etc.)' 'co' 'sweden' 'singapore'
 'san diego, ca' 'australia' 'switzerland' 'benelux' 'netherlands'
 'remote - us' 'research triangle park, nc' 'poland' 'argentine' 'austria'
 'boston' 'kenya' 'tennessee' 'nl']
[nan 'canada' 'us' 'france' 'uk' 'denmark' 'germany' 'spain' 'benelux'
 'eu' 'india' 'sweden' 'singapore' 'australia' 'switzerland' 'poland'
 'argentine' 'austria' 'kenya']


In [26]:
# Merge two columns reporting the country:
df["country"] = df["country_1"].fillna(df["country_2"])

# Inspect how many lack this information:
df.loc[df["country"].isna()]

,Timestamp,country_1,city,company_field,company_name,company_type,company_size,role,degrees_2,List other relevant and recognized certifications,...,edu_BSc,edu_HS,edu_JD,edu_M.D./PharmD/D.D.S.,edu_MD,edu_MSc,edu_PhD,edu_PharmD,edu_highest,country
974,1/14/2024 6:08:26,NaN,NaN,Food,Prefer not to say,Public,NaN,1,NaN,NaN,...,0,0,0,0,0,0,1,0,PhD,NaN
1401,4/17/2024 4:07:20,NaN,NaN,Contract Research Organization,Prefer not to say,Private,200-1000,Bioinformatics engineer,NaN,NaN,...,0,0,0,0,0,1,0,0,MSc,NaN
1608,5/18/2024 19:11:41,NaN,NaN,Consulting,Prefer not to say,Private,1-50,Principal,NaN,NaN,...,1,0,0,0,0,0,0,0,BSc,NaN


In [27]:
df["country"].unique()

array(['canada', 'germany', 'slovenia', 'uk', 'us', 'austria', 'india',
       'ireland', 'benelux', 'america', 'australia', 'portugal', 'sweden',
       'china', 'greece', 'hungary', 'switzerland', 'france', 'denmark',
       'spain', 'eu', nan, 'singapore', 'poland', 'argentine', 'kenya'],
      dtype=object)

In [28]:
# Label countries in Europe:
non_eu_countries = ["us", "china", "india", "kenya", "australia", "argentine", "singapore", "canada"]

df["region"] = "others"
df.loc[ (~df["country"].isin(non_eu_countries)), "region"] = "europe"
df.loc[ df["country"] == "us", "region"] = "us"
df

,Timestamp,country_1,city,company_field,company_name,company_type,company_size,role,degrees_2,List other relevant and recognized certifications,...,edu_HS,edu_JD,edu_M.D./PharmD/D.D.S.,edu_MD,edu_MSc,edu_PhD,edu_PharmD,edu_highest,country,region
0,8/6/2023 13:35:24,canada,Toronto,Academia/Research/Institution,Prefer not to say,Private,200-1000,Medical Writer,Bachelors or Equivalent,NaN,...,0,0,0,0,0,0,0,BSc,canada,others
1,4/19/2023 19:24:24,germany,Munich,Academia/Research/Institution,Max Planck Institute,Academia/College/Universities,5000+,Post-doc,PhD or Equivalent,NaN,...,0,0,0,0,0,1,0,PhD,germany,europe
2,6/22/2023 16:28:50,slovenia,Ljubljana,Academia/Research/Institution,Prefer not to say,Public,50-200,Student assistant researcher,Bachelors or Equivalent,NaN,...,0,0,0,0,0,0,0,BSc,slovenia,europe
3,6/14/2023 15:30:45,uk,Edinburgh,Academia/Research/Institution,University of Edinburgh,Academia/College/Universities,5000+,Postdoc,PhD or Equivalent,NaN,...,0,0,0,0,0,1,0,PhD,uk,europe
4,3/29/2023 2:06:32,us,Bay Area,Academia/Research/Institution,Prefer not to say,Start-up,1-50,Research Scientist,Masters or Equivalent,NaN,...,0,0,0,0,1,0,0,MSc,us,us
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023,9/29/2024 21:11:18,NaN,NaN,Big Pharmaceuticals,Prefer not to say,Public,5000+,Asset Exec,NaN,NaN,...,0,0,0,1,0,0,0,Other,us,us
2024,9/29/2024 22:12:46,NaN,NaN,Contraceptives,Mucommune,Start-up,1-50,Scientist,NaN,NaN,...,0,0,0,0,0,0,0,BSc,us,us
2025,9/30/2024 5:33:54,NaN,NaN,Agriculture,Syngenta,Private,5000+,Principal Scientist I,NaN,NaN,...,0,0,0,0,0,1,0,PhD,uk,europe
2026,9/30/2024 8:01:19,NaN,NaN,Big Pharmaceuticals,Regeneron,Public,5000+,Associate Scientist,NaN,NaN,...,0,0,0,0,0,0,0,BSc,us,us


#### Company type

In [29]:
# Inspect reported industry names
df_type_counts = df.groupby(["company_field"]).count()["Timestamp"]
df_type_counts.sort_values(ascending=False).to_frame().head(30)

,Timestamp
company_field,
Big Pharmaceuticals,534
Industrial Biotech,414
Pharmaceutical (including “big pharma”),403
Academia/Research/Institution,114
Contract Research Organization,86
Contract Manufacturing Organization,70
Medical/Health Devices,63
Biotech Equipment,60
Medical Devices,31


In [30]:
# Select uncommon/non-standard industry names:
nonstd_labels = df_type_counts.loc[df_type_counts < 5].index.to_list()
print(nonstd_labels)

['Analytical Equipment', 'Ancillary reagents', 'Bioengineered organs', 'Biologics', 'Biotech ', 'Biotech Software', 'Biotech VC', 'Biotech software', 'Biotech start-up', 'Biotech startup', 'Biotech startup ', 'CDMO', 'CDMO CGMP manufacturing (large molecule)', 'Cdmo', 'Cell & Gene Therapy Materials', 'Cell Therapy', 'Cell and Gene', 'Cell and Gene Therapy', 'Cell culture/separation', 'Cell therapy', 'Climate', 'Clinical Diagnostics', 'Clinical Diagnostics ', 'Conseil', 'Conservation', 'Consulting', 'Consulting ', 'Consumables', 'Contraceptives', 'Contract Testing Organization', 'Contract testing lab', 'Cosmetics', 'Diagnostics ', 'Dna synthesis ', 'Drug Discovery', 'Drug Discovery Startup', 'Dx and Tx', 'Energy', 'Gene Therapy', 'Gene therapy', 'Genetic Testing', 'Genome Editing', 'Genomics', 'Health insurer/HMO', 'Hospital', 'IT', 'Immuno oncology', 'Informatics', 'Instrumentation', 'Lab supply distribution ', 'Life Sciences Software', 'Life Sciences Tools Company', 'Liquid Biopsy', '

In [31]:
# Lump all into one category:
df["company_field"] = df["company_field"].replace(nonstd_labels, "Others")
df.head()

,Timestamp,country_1,city,company_field,company_name,company_type,company_size,role,degrees_2,List other relevant and recognized certifications,...,edu_HS,edu_JD,edu_M.D./PharmD/D.D.S.,edu_MD,edu_MSc,edu_PhD,edu_PharmD,edu_highest,country,region
0,8/6/2023 13:35:24,canada,Toronto,Academia/Research/Institution,Prefer not to say,Private,200-1000,Medical Writer,Bachelors or Equivalent,NaN,...,0,0,0,0,0,0,0,BSc,canada,others
1,4/19/2023 19:24:24,germany,Munich,Academia/Research/Institution,Max Planck Institute,Academia/College/Universities,5000+,Post-doc,PhD or Equivalent,NaN,...,0,0,0,0,0,1,0,PhD,germany,europe
2,6/22/2023 16:28:50,slovenia,Ljubljana,Academia/Research/Institution,Prefer not to say,Public,50-200,Student assistant researcher,Bachelors or Equivalent,NaN,...,0,0,0,0,0,0,0,BSc,slovenia,europe
3,6/14/2023 15:30:45,uk,Edinburgh,Academia/Research/Institution,University of Edinburgh,Academia/College/Universities,5000+,Postdoc,PhD or Equivalent,NaN,...,0,0,0,0,0,1,0,PhD,uk,europe
4,3/29/2023 2:06:32,us,Bay Area,Academia/Research/Institution,Prefer not to say,Start-up,1-50,Research Scientist,Masters or Equivalent,NaN,...,0,0,0,0,1,0,0,MSc,us,us


In [32]:
df.groupby("company_field").count()["Timestamp"].sort_values(ascending=False).to_frame()

,Timestamp
company_field,
Big Pharmaceuticals,534
Industrial Biotech,414
Pharmaceutical (including “big pharma”),403
Others,131
Academia/Research/Institution,114
Contract Research Organization,86
Contract Manufacturing Organization,70
Medical/Health Devices,63
Biotech Equipment,60


#### Role

In [33]:
df["role"].unique()

array(['Medical Writer', 'Post-doc', 'Student assistant researcher',
       'Postdoc', 'Research Scientist', 'Bioinformatics intern',
       'Technical Application Scientist II',
       'Process development associate II', 'Licensing Agent',
       'Associate Scientist II', 'Research Associate', 'Lab Technician',
       'Associate Scientist ', 'Supervisor', 'Bioinformatics Analyst II',
       'Scientist I', 'Director', 'Research Services Technician',
       'Molecular Biology Research Technician', 'Associate Scientist',
       'Research Associate I', 'Supervisor I',
       'Senior Research Associate ', 'Senior Data Scientist ',
       'Senior Research Technician', 'Quality Assurance Associate II',
       'Institute Associate Scientist II', 'Research Assistant',
       'Res Asst Prof', 'Lab Technician ', 'Scientific research lead',
       'Research Associate II', 'Manager', 'Senior Scientist /Team Lead',
       'Research Associate III', 'Associate scientist ', 'Biologist',
       'Scient

In [34]:
repl_role = {".": "",
             "post doc": "postdoc",
             "post-doc": "postdoc",
             "postdoctoral scientist": "postdoc",
             "postdoctoral fellow": "postdoc",
             "postdoctoral researcher": "postdoc",
             "phd student": "phd",
             "phd candidate": "phd",
             "senior": "sr",
             "quality control": "qc",
             "3": "iii",              
             "2": "ii",
             "ll": "ii",
             "1": "i",
             }

df["role"] = df["role"].str.lower()
df["role"] = df["role"].str.rstrip(" ").str.strip(" ")
print(df["role"] .unique())

df = utils.value_replacement(df, replacement=repl_role, column="role")
print(df["role"] .unique())

['medical writer' 'post-doc' 'student assistant researcher' 'postdoc'
 'research scientist' 'bioinformatics intern'
 'technical application scientist ii' 'process development associate ii'
 'licensing agent' 'associate scientist ii' 'research associate'
 'lab technician' 'associate scientist' 'supervisor'
 'bioinformatics analyst ii' 'scientist i' 'director'
 'research services technician' 'molecular biology research technician'
 'research associate i' 'supervisor i' 'senior research associate'
 'senior data scientist' 'senior research technician'
 'quality assurance associate ii' 'institute associate scientist ii'
 'research assistant' 'res asst prof' 'scientific research lead'
 'research associate ii' 'manager' 'senior scientist /team lead'
 'research associate iii' 'biologist' 'scientist' 'microbiologist'
 'sr scientist' 'technical support scientist' 'sr. technician'
 'operations scientist' 'bioprocessing tech'
 'sr field applications scientist' 'product marketing manager' 'vp'
 'se

In [35]:
df.groupby("role").count()["Timestamp"].sort_values().to_frame().tail(60)

,Timestamp
role,
qc specialist,3
research scientist i,3
software engineer,3
qc analyst ii,3
"specialist, clinical operations",3
sr automation engineer,3
process development associate,3
sr computational biologist,3
operations manager,3


In [36]:
# Add new labels:
df["seniority"] = "normal"
df.loc[df["role"].str.contains("sr "), "seniority"] = "senior"


correspondence = {"director": "director",
                  "manager": "manager",
                  "bioinf": "bioinformatics",
                  "technician": "technician",
                  "research": "scientist",   # <<<<
                  "postdoc": "scientist",
                  "phd": "scientist",
                  "scientist": "scientist",
                  "engineer": "engineer",
                  "data": "data",
}

# Initialize column with undefined role category:
df["role_class"] = "other"

# Replace based on the strings found:
for role, role_class in correspondence.items():
    df.loc[df["role"].str.contains(role), "role_class"] = role_class

df

,Timestamp,country_1,city,company_field,company_name,company_type,company_size,role,degrees_2,List other relevant and recognized certifications,...,edu_M.D./PharmD/D.D.S.,edu_MD,edu_MSc,edu_PhD,edu_PharmD,edu_highest,country,region,seniority,role_class
0,8/6/2023 13:35:24,canada,Toronto,Academia/Research/Institution,Prefer not to say,Private,200-1000,medical writer,Bachelors or Equivalent,NaN,...,0,0,0,0,0,BSc,canada,others,normal,other
1,4/19/2023 19:24:24,germany,Munich,Academia/Research/Institution,Max Planck Institute,Academia/College/Universities,5000+,postdoc,PhD or Equivalent,NaN,...,0,0,0,1,0,PhD,germany,europe,normal,scientist
2,6/22/2023 16:28:50,slovenia,Ljubljana,Academia/Research/Institution,Prefer not to say,Public,50-200,student assistant researcher,Bachelors or Equivalent,NaN,...,0,0,0,0,0,BSc,slovenia,europe,normal,scientist
3,6/14/2023 15:30:45,uk,Edinburgh,Academia/Research/Institution,University of Edinburgh,Academia/College/Universities,5000+,postdoc,PhD or Equivalent,NaN,...,0,0,0,1,0,PhD,uk,europe,normal,scientist
4,3/29/2023 2:06:32,us,Bay Area,Academia/Research/Institution,Prefer not to say,Start-up,1-50,research scientist,Masters or Equivalent,NaN,...,0,0,1,0,0,MSc,us,us,normal,scientist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023,9/29/2024 21:11:18,NaN,NaN,Big Pharmaceuticals,Prefer not to say,Public,5000+,asset exec,NaN,NaN,...,0,1,0,0,0,Other,us,us,normal,other
2024,9/29/2024 22:12:46,NaN,NaN,Others,Mucommune,Start-up,1-50,scientist,NaN,NaN,...,0,0,0,0,0,BSc,us,us,normal,scientist
2025,9/30/2024 5:33:54,NaN,NaN,Agriculture,Syngenta,Private,5000+,principal scientist i,NaN,NaN,...,0,0,0,1,0,PhD,uk,europe,normal,scientist
2026,9/30/2024 8:01:19,NaN,NaN,Big Pharmaceuticals,Regeneron,Public,5000+,associate scientist,NaN,NaN,...,0,0,0,0,0,BSc,us,us,normal,scientist


### Save data

In [37]:
df_raw.to_excel("data/data_raw.xlsx")
df.to_excel("data/data_preprocessed.xlsx")